05 tf-idfを用いたデータのスケール変換
==============================

* `tf-idf`：特徴量がどの程度情報を持っていそうかに応じて、特徴量のスケールを変換する

    * この手法は、特定の文書にだけ頻繁に現れる単語に大きな重みを与え、コーパス中の多数の文書に現れる単語にはあまり重みを与えない
    
    * 特定の文書にだけ頻出し、他の文書にはあまり現れない単語は、その文書をよく示しているのではないか、という発想
    
* scikit-learnは`tf-idf`を2つのクラスで実装している

    * `TfidfTransformer`は`CountVectorizer`の生成する疎行列を入力とする
    
    * `TfidfVectorizer`はテキストデータを入力とし、BoW特徴量抽出とtf-idf変換を行う
    
    * tf-idfスケール変換には、様々な方法がある
    
    * `TfidTransformer`でも`TfidVectorizer`でも、文書$d$における、単語$w$のtf-idfスコアは下のように与えられる

* $N$：訓練セットの文書の数

* $N_w$：訓練セット中の$w$が現れる文書の数

* $tf$：対象の文書$d$(変換を行う文書)中に$w$が現れる回数

\begin{eqnarray}
tfidf(w, d) = tf(log(\frac{N + 1}{N_w + 1}) + 1)
\end{eqnarray}

* 2つのクラスはいずれも、tf-idf表現を計算したあとでL2正則化を行う

    * つまり、それぞれの文書の表現の長さが、ユークリッド長で1になるようにスケール変換を行う
    
    * このように変換すると、文書の長さ(単語数)がベクトル表現に影響を与えなくなる

* tf-idfは訓練データの統計的性質を利用するので、パイプラインを用いてグリッドサーチの結果が有効になるようにする

In [1]:
from sklearn.datasets import load_files

reviews_train = load_files("/Users/MacUser/data/aclImdb/train/")
text_train, y_train = reviews_train.data, reviews_train.target
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]
reviews_test = load_files("/Users/MacUser/data/aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)

import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
scores = cross_val_score(LogisticRegression(), X_train, y_train, cv=5)

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

X_test = vect.transform(text_test)

vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(TfidfVectorizer(min_df=5, norm=None),
                     LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))

/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Best cross-validation score: 0.89


* この場合には、tf-idf変換を行っても性能は向上しなかった

    * tf-idfを用いると性能が少し良くなる
    
    * また、tf-idfがどの単語が最も重要だと判断したかをみることもできる
    
    * tf-idfによるスケール変換は、文書を区別するためのものだが、純粋に教師なしの手法であることに留意する
    
    * このため、ここでの「重要」さは、本来の興味の対象である「肯定的なレビュー」「否定的なレビュー」のラベルには必ずしも関係ない
    
* まず、パイプラインから`TfidfVectorizer`を取り出す

In [4]:
vectorizer = grid.best_estimator_.named_steps["tfidfvectorizer"]
# 訓練データセットを変換
X_train = vectorizer.transform(text_train)
# それぞれの特徴量のデータセット中での最大値を見つける
max_value = X_train.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
# 特徴量名を取得
feature_names = np.array(vectorizer.get_feature_names())

print("Features with lowest tfidf:\n{}".format(
      feature_names[sorted_by_tfidf[:20]]))

print("Features with highest tfidf: \n{}".format(
      feature_names[sorted_by_tfidf[-20:]]))

Features with lowest tfidf:
['poignant' 'disagree' 'instantly' 'importantly' 'lacked' 'occurred'
 'currently' 'altogether' 'nearby' 'undoubtedly' 'directs' 'fond'
 'stinker' 'avoided' 'emphasis' 'commented' 'disappoint' 'realizing'
 'downhill' 'inane']
Features with highest tfidf: 
['coop' 'homer' 'dillinger' 'hackenstein' 'gadget' 'taker' 'macarthur'
 'vargas' 'jesse' 'basket' 'dominick' 'the' 'victor' 'bridget' 'victoria'
 'khouri' 'zizek' 'rob' 'timon' 'titanic']


* tf-idfが低い特徴量は、多くの文書に共通して出現するか、あまり出現しないか、もしくは非常に長い文書にしか出現しないか、である

    * tf-idfの高い特徴量の多くは、特定の映画を指している
    
    * これらの単語は、センチメント分析タスクにはあまり役に立たない

* 文書頻度の逆数(idf)が小さい単語を見つけることもできる

    * このような単語は高い頻度で現れるため、重要でないと考えられる単語
    
    * 訓練セットに対する文書頻度の逆数は`idf_`属性に格納されている

In [5]:
sorted_by_idf = np.argsort(vectorizer.idf_)
print("Features with lowest idf:\n{}".format(
       feature_names[sorted_by_idf[:100]]))

Features with lowest idf:
['the' 'and' 'of' 'to' 'this' 'is' 'it' 'in' 'that' 'but' 'for' 'with'
 'was' 'as' 'on' 'movie' 'not' 'have' 'one' 'be' 'film' 'are' 'you' 'all'
 'at' 'an' 'by' 'so' 'from' 'like' 'who' 'they' 'there' 'if' 'his' 'out'
 'just' 'about' 'he' 'or' 'has' 'what' 'some' 'good' 'can' 'more' 'when'
 'time' 'up' 'very' 'even' 'only' 'no' 'would' 'my' 'see' 'really' 'story'
 'which' 'well' 'had' 'me' 'than' 'much' 'their' 'get' 'were' 'other'
 'been' 'do' 'most' 'don' 'her' 'also' 'into' 'first' 'made' 'how' 'great'
 'because' 'will' 'people' 'make' 'way' 'could' 'we' 'bad' 'after' 'any'
 'too' 'then' 'them' 'she' 'watch' 'think' 'acting' 'movies' 'seen' 'its'
 'him']


* 予想された通り、これらのほとんどの英語のストップワードである

* いくつかは明らかに、映画レビューに固有の単語である

* "good"、"great"などの単語は、センチメント分析タスクには非常に重要だと思われているのにも関わらず、多くの文書に頻出する
    
    * tf-idfの尺度では、「最も関連性が低い」と判断されてしまっている

| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/04/06 |